In [ ]:
install.packages('devtools')
devtools::install_github('PheWAS/PheWAS')

In [ ]:
library(PheWAS)

In [ ]:
# Input and output data
PHENO="/path/to/PheWAS/input/phenotypes.csv" # Use the PheWAS_ICD_input.csv output of script 3_Data preparation/UKB/7_gather_phewas.ipynb
COV="/path/to/PheWAS/input/phenotypes.csv" # Use the PheWAS_covariates.csv output of script 3_Data preparation/UKB/7_gather_phewas.ipynb
GENO="/path/to/PheWAS/input/phenotypes.csv" # Use the PheWAS_genotype.csv output of script 3_Data preparation/UKB/7_gather_phewas.ipynb

OUTPUT_DIR="/path/to/output/directory"
# Outputs will be (1) two CSV files with PheWAS results, one with and one without phecode annotations. The table with annotations is presented in Table S5J
# and (2) a PDF of the resulting Manhattan plot as presented in Fig S5B

In [ ]:
# Load data
pheno <- read.csv(PHENO)
cov <- read.csv(COV)
geno <- read.csv(GENO)

In [ ]:
# Update some annotations
pheno$vocabulary_id <- 'ICD10'
pheno$index <- 1
pheno <- pheno[, c('id', 'vocabulary_id', 'code', 'index')]
pheno$id <- as.character(pheno$id)
cov[cov$sex==1, 'sex'] <- 'M'
cov[cov$sex==0, 'sex'] <- 'F'
id_sex <- cov[, c('id', 'sex')]
id_sex$id <- as.character(id_sex$id)

In [ ]:
# Map ICD10 codes to phecodes
phe_df <- createPhenotypes(pheno, min.code.count=1, id.sex=id_sex,
                           add.phecode.exclusions=T, translate=T, vocabulary.map=PheWAS::phecode_map_icd10)
# Remove any duplicate rows
phe_df <- phe_df[!duplicated(phe_df$id), ]

In [ ]:
# Merge data
dat <- merge(cov, geno, by='id')
dat <- merge(dat, phe_df, by='id')

phenos <- colnames(phe_df)
phenos <- phenos[phenos!='id']

In [ ]:
# Run PheWAS
cores <- as.integer(detectCores()/2)
res <- phewas(data=dat, phenotypes=phenos, genotypes='X16p12_del',
                  covariates=c('yob', 'sex', 'PC1', 'PC2', 'PC3', 'PC4'),
                  min.records=20, additive.genotypes=F, cores=cores)

In [ ]:
# Save results
write.csv(res, 'PheWAS_res.csv', quote=T, row.names=F)
system(paste('dx upload PheWAS_res.csv --path', OUTPUT_DIR, '/PheWAS_res.csv', sep=''))

In [ ]:
res$phenotype <- gsub('X', '', res$phenotype)

In [ ]:
# Annotate Phewas descriptions
res_d <- addPhecodeInfo(res)

# Order and save
res_d <- res_d[order(res_d$p),]
print(res_d[1:10, 1:10])
write.csv(res_d, '2_PheWAS_annotated_results.csv', quote=T, row.names=F)

system(paste('dx upload PheWAS_annotated_results.csv --path', OUTPUT_DIR, '/PheWAS_annotated_results.csv', sep=''))

In [ ]:
# Make Manhattan plot
pdf("UKB_PheWAS_Manhattan.pdf", width=10, height=5)
phewasManhattan(res, title='UKB 16p12.1 deletion PheWAS')
dev.off()

In [ ]:
# Upload output
system(paste('dx upload UKB_PheWAS_Manhattan.pdf --path', OUTPUT_DIR, '/UKB_PheWAS_Manhattan.pdf', sep=''))